In [3]:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
import pickle
import numpy as np
import os

In [17]:
file = open("Self.txt", "r", encoding = "utf8")

# store file in list
lines = []
for i in file:
    lines.append(i)
 
# Convert list to string
data = ""
for i in lines:
  data = ' '. join(lines) 
 
#replace unnecessary stuff with space
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','').replace(',','').replace(';','').replace('-','')  #new line, carriage return, unicode character --> replace by space
 
#remove unnecessary spaces 
data = data.split()
data = ' '.join(data)
data[:500]

'If only I had more selfcontrol. I don’t have such an iron mind. I want to enjoy life too — not just suffer. These are comments I might get when people learn about my lifestyle. I’m one of those annoying people who eats lots of fruits and vegetables exercises five times a week saves a portion of their salary and writes or reads every morning before work — I have good selfcontrol. What’s more retaining this lifestyle doesn’t feel particularly difficult to me I don’t grit my teeth to avoid unhealth'

In [18]:
len(data)

11779

In [19]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
 
# saving the tokenizer for predict function
pickle.dump(tokenizer, open('token.pkl', 'wb'))
 
sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]
len(sequence_data)
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)
sequences = []
 
for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequences.append(words)
     
print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]
X = []
y = []
 
for i in sequences:
    X.append(i[0:3])
    y.append(i[3])
     
X = np.array(X)
y = np.array(y)
print("Data: ", X[:10])
print("Response: ", y[:10])
y = to_categorical(y, num_classes=vocab_size)
y[:5]

680
The Length of sequences are:  1883
Data:  [[ 31 106  14]
 [106  14 268]
 [ 14 268  15]
 [268  15   3]
 [ 15   3  14]
 [  3  14  76]
 [ 14  76  12]
 [ 76  12 151]
 [ 12 151  53]
 [151  53 269]]
Response:  [268  15   3  14  76  12 151  53 269 270]


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [20]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 3, 10)             6800      
                                                                 
 lstm_2 (LSTM)               (None, 3, 1000)           4044000   
                                                                 
 lstm_3 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense_2 (Dense)             (None, 1000)              1001000   
                                                                 
 dense_3 (Dense)             (None, 680)               680680    
                                                                 
Total params: 13736480 (52.40 MB)
Trainable params: 13736480 (52.40 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
 #from tensorflow import keras
#from keras.utils import plot_model
#keras.utils.plot_model(model, to_file='plot.png', show_layer_names=True)

# Gives ERROR 

In [22]:
 
checkpoint = ModelCheckpoint("next_words.h5", monitor='loss', verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
model.fit(X, y, epochs=75, batch_size=64, callbacks=[checkpoint])


Epoch 1/75
30/30 [==============================] - ETA: 0s - loss: 6.2739 - accuracy: 0.0281
Epoch 1: loss improved from inf to 6.27390, saving model to next_words.h5
30/30 [==============================] - 8s 151ms/step - loss: 6.2739 - accuracy: 0.0281
Epoch 2/75


c:\Users\Vedant\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


30/30 [==============================] - ETA: 0s - loss: 5.9232 - accuracy: 0.0335
Epoch 2: loss improved from 6.27390 to 5.92321, saving model to next_words.h5
30/30 [==============================] - 5s 153ms/step - loss: 5.9232 - accuracy: 0.0335
Epoch 3/75
30/30 [==============================] - ETA: 0s - loss: 5.8436 - accuracy: 0.0393
Epoch 3: loss improved from 5.92321 to 5.84358, saving model to next_words.h5
30/30 [==============================] - 4s 149ms/step - loss: 5.8436 - accuracy: 0.0393
Epoch 4/75
30/30 [==============================] - ETA: 0s - loss: 5.7725 - accuracy: 0.0393
Epoch 4: loss improved from 5.84358 to 5.77249, saving model to next_words.h5
30/30 [==============================] - 6s 189ms/step - loss: 5.7725 - accuracy: 0.0393
Epoch 5/75
30/30 [==============================] - ETA: 0s - loss: 5.7227 - accuracy: 0.0361
Epoch 5: loss improved from 5.77249 to 5.72272, saving model to next_words.h5
30/30 [==============================] - 5s 183ms/step -

In [23]:

# Load the model and tokenizer
model = load_model('next_words.h5')
tokenizer = pickle.load(open('token.pkl', 'rb'))
 
def Predict_Next_Words(model, tokenizer, text):
 
  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array(sequence)
  preds = np.argmax(model.predict(sequence))
  predicted_word = ""
   
  for key, value in tokenizer.word_index.items():
      if value == preds:
          predicted_word = key
          break
   
  print(predicted_word)
  return predicted_word
while(True):
  text = input("Enter your line: ")
   
  if text == "0":
      print("Execution completed.....")
      break
   
  else:
      try:
          text = text.split(" ")
          text = text[-3:]
          print(text)
         
          Predict_Next_Words(model, tokenizer, text)
           
      except Exception as e:
        print("Error occurred: ",e)
        continue

['this', 'temptation-avoidance']
1/1 [==============================] - 1s 914ms/step
as
['of', 'the', 'everyday']
1/1 [==============================] - 1s 951ms/step
temptation
['scored', 'high', 'on']
1/1 [==============================] - 0s 30ms/step
trait
['are', 'like', 'having']
1/1 [==============================] - 0s 30ms/step
researchers
Execution completed.....
